In [34]:
import os

In [35]:
from langchain_community.document_loaders import PyMuPDFLoader

In [36]:
loader = PyMuPDFLoader("industry_laws.pdf")
pages = loader.load()

In [37]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [38]:
# 문서를 문장으로 분리
## 청크 크기 500, 각 청크의 50자씩 겹치도록 청크를 나눈다
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
)
docs = text_splitter.split_documents(pages)

In [39]:
from langchain.embeddings import HuggingFaceEmbeddings

In [40]:
# 문장을 임베딩으로 변환하고 벡터 저장소에 저장
embeddings = HuggingFaceEmbeddings(
    model_name='BAAI/bge-m3',
    model_kwargs={'device':'cuda'},
    encode_kwargs={'normalize_embeddings':True},
)

In [41]:
# 벡터 저장소 생성
from langchain.vectorstores import Chroma
vectorstore = Chroma.from_documents(docs, embeddings)


# 벡터 저장소 경로 설정
## 현재 경로에 'vectorstore' 경로 생성
vectorstore_path = 'vectorstore'
os.makedirs(vectorstore_path, exist_ok=True)

# 벡터 저장소 생성 및 저장
vectorstore = Chroma.from_documents(docs, embeddings, persist_directory=vectorstore_path)
# 벡터스토어 데이터를 디스크에 저장
vectorstore.persist()
print("Vectorstore created and persisted")

Vectorstore created and persisted


In [42]:
from langchain_community.chat_models import ChatOllama

# Ollama 를 이용해 로컬에서 LLM 실행
## llama3-ko-instruct 모델 다운로드는 Ollama 사용법 참조
model = ChatOllama(model="llama3.1", temperature=0)

In [43]:
retriever = vectorstore.as_retriever(search_kwargs={'k': 5})

In [53]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate


# Prompt 템플릿 생성
template = '''
상대방의 요청에 최대한 정확하고 객관적으로 답하자. 
모든 대답은 한국어(Korean)으로 대답하자.
사고 경위가 입력되면 그 사고 경위를 분석하고,
사고경위와 관련된 조문과 조항을 "산업안전보건기준에 관학 규칙(약칭: 안전보건규칙)"에서 5개를 찾는다.
그리고 사고경위와 관련이 높은 순서대로 나열해서 관련된 조문과 조항을 보고서 형식으로 언급하자 :

{context}
Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return '\n\n'.join([d.page_content for d in docs])

# RAG Chain 연결
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [56]:
# Chain 실행
query = "사다리를 놓고 조명을 설치하러 올라가다가 사다리가 미끄러지면서 2M 높이에서 추락함."
answer = rag_chain.invoke(query)
print("Query:", query)
print("Answer:\n", answer)

Query: 사다리를 놓고 조명을 설치하러 올라가다가 사다리가 미끄러지면서 2M 높이에서 추락함.
Answer:
 **사고 경위 분석**

*   사다리 사용 시 다른 근로자가 지지하여 넘어지지 않도록 할 것(제7조 제1항)
*   이동식 사다리의 최대사용하중을 초과하지 않는 범위 내에서만 사용할 것(제7조 제2항)
*   작업 높이가 2미터 이상인 경우에는 안전모와 안전대를 함께 착용할 것(제7조 제6항)

**관련 조문 및 조항**

1.  **제43조(개구부 등의 방호 조치)**: 사업주는 작업발판 및 통로의 끝이나 개구부로서 근로자가 추락할 위험이 있는 장소에는 안전난간, 울타리, 수직형 추락방망 또는 덮개 등(이하 이 조에서 “난간등”이라 한다)의 방호 조치를 충분한 강도와 구조를 갖추어야 하며, 난간등의 설치 및 관리는 근로자의 안전을 고려하여 할 것.
2.  **제7조(사다리 사용)**: 사업주는 사다리를 사용할 때에는 다음 각 호의 사항을 준수해야 한다.
    *   사다리를 다른 근로자가 지지하여 넘어지지 않도록 할 것
    *   이동식 사다리의 제조사가 정하여 표시한 이동식 사다리의 최대사용하중을 초과하지 않는 범위 내에서만 사용할 것
    *   이동식 사다리를 설치한 바닥면에서 높이 3.5미터 이하의 장소에서만 작업할 것
    *   이동식 사다리의 최상부 발판 및 그 하단 디딤대에 올라서서 작업하지 않을 것. 다만, 높이 1미터 이하의 사다리는 제외한다.
    *   안전모를 착용하되, 작업 높이가 2미터 이상인 경우에는 안전모와 안전대를 함께 착용할 것
    *   이동식 사다리 사용 전 변형 및 이상 유무 등을 점검하여 이상이 발견되면 즉시 수리하거나 그 밖에 필요한 조치를 할 것

**사고의 원인**

*   사다리가 미끄러지면서 2M 높이에서 추락함. 이는 사다리의 안전성과 사용자의 주의가 부족한 결과로 발생한 것으로 보임.

**예방조치**

*   사다리 사용 시 다른 근로자가 지지하여 넘어지지 않도록 할 것
* 